Benson = EGUB/03658

51.6201, -1.0983

In [1]:
import datetime as dt
import os

import requests
import xarray as xr
from dask.distributed import wait, Client as DaskClient
# from ecmwf.opendata import Client as ECMWFClient

## Dynamical - GFS

In [26]:
ds_gfs = xr.open_zarr("https://data.dynamical.org/noaa/gfs/forecast/latest.zarr?email=eleanor.tracey@hotmail.co.uk")

ds_gfs

<xarray.Dataset> Size: 119TB
Dimensions:                                     (init_time: 6551,
                                                 lead_time: 209, latitude: 721,
                                                 longitude: 1440)
Coordinates:
  * init_time                                   (init_time) datetime64[ns] 52kB ...
  * lead_time                                   (lead_time) timedelta64[ns] 2kB ...
  * latitude                                    (latitude) float64 6kB 90.0 ....
  * longitude                                   (longitude) float64 12kB -180...
    expected_forecast_length                    (init_time) timedelta64[ns] 52kB dask.array<chunksize=(6551,), meta=np.ndarray>
    ingested_forecast_length                    (init_time) timedelta64[ns] 52kB dask.array<chunksize=(6551,), meta=np.ndarray>
    spatial_ref                                 int64 8B ...
    valid_time                                  (init_time, lead_time) datetime64[ns] 11MB dask.array<chunksize=(6551, 209), meta=np.ndarray>
Data variables: (12/21)
    categorical_freezing_rain_surface           (init_time, lead_time, latitude, longitude) float32 6TB dask.array<chunksize=(1, 105, 121, 121), meta=np.ndarray>
    categorical_ice_pellets_surface             (init_time, lead_time, latitude, longitude) float32 6TB dask.array<chunksize=(1, 105, 121, 121), meta=np.ndarray>
    categorical_rain_surface                    (init_time, lead_time, latitude, longitude) float32 6TB dask.array<chunksize=(1, 105, 121, 121), meta=np.ndarray>
    categorical_snow_surface                    (init_time, lead_time, latitude, longitude) float32 6TB dask.array<chunksize=(1, 105, 121, 121), meta=np.ndarray>
    downward_long_wave_radiation_flux_surface   (init_time, lead_time, latitude, longitude) float32 6TB dask.array<chunksize=(1, 105, 121, 121), meta=np.ndarray>
    downward_short_wave_radiation_flux_surface  (init_time, lead_time, latitude, longitude) float32 6TB dask.array<chunksize=(1, 105, 121, 121), meta=np.ndarray>
    ...                                          ...
    temperature_2m                              (init_time, lead_time, latitude, longitude) float32 6TB dask.array<chunksize=(1, 105, 121, 121), meta=np.ndarray>
    total_cloud_cover_atmosphere                (init_time, lead_time, latitude, longitude) float32 6TB dask.array<chunksize=(1, 105, 121, 121), meta=np.ndarray>
    wind_u_100m                                 (init_time, lead_time, latitude, longitude) float32 6TB dask.array<chunksize=(1, 105, 121, 121), meta=np.ndarray>
    wind_u_10m                                  (init_time, lead_time, latitude, longitude) float32 6TB dask.array<chunksize=(1, 105, 121, 121), meta=np.ndarray>
    wind_v_100m                                 (init_time, lead_time, latitude, longitude) float32 6TB dask.array<chunksize=(1, 105, 121, 121), meta=np.ndarray>
    wind_v_10m                                  (init_time, lead_time, latitude, longitude) float32 6TB dask.array<chunksize=(1, 105, 121, 121), meta=np.ndarray>
Attributes:
    dataset_id:           noaa-gfs-forecast
    dataset_version:      0.2.7
    name:                 NOAA GFS forecast
    description:          Weather forecasts from the Global Forecast System (...
    attribution:          NOAA NWS NCEP GFS data processed by dynamical.org f...
    spatial_domain:       Global
    spatial_resolution:   0.25 degrees (~20km)
    time_domain:          Forecasts initialized 2021-05-01 00:00:00 UTC to Pr...
    time_resolution:      Forecasts initialized every 6 hours
    forecast_domain:      Forecast lead time 0-384 hours (0-16 days) ahead
    forecast_resolution:  Forecast step 0-120 hours: hourly, 123-384 hours: 3...

In [27]:
variable_list = [
    "pressure_reduced_to_mean_sea_level",
    "relative_humidity_2m", 
    "temperature_2m", 
    "total_cloud_cover_atmosphere", 
    "wind_u_10m",
    "wind_v_10m"
    ]

temp_var_list = [
    "pressure_reduced_to_mean_sea_level", 
    "relative_humidity_2m", 
    "total_cloud_cover_atmosphere"
]

ds_gfs_subset = ds_gfs[temp_var_list].sel(
    latitude=slice(52, 51.25), longitude=slice(-1.5, -0.75), init_time=slice("2022-10-24", "2025-10-24")
    )
ds_gfs_subset

<xarray.Dataset> Size: 183MB
Dimensions:                             (init_time: 4387, lead_time: 209,
                                         latitude: 4, longitude: 4)
Coordinates:
  * init_time                           (init_time) datetime64[ns] 35kB 2022-...
  * lead_time                           (lead_time) timedelta64[ns] 2kB 00:00...
  * latitude                            (latitude) float64 32B 52.0 ... 51.25
  * longitude                           (longitude) float64 32B -1.5 ... -0.75
    expected_forecast_length            (init_time) timedelta64[ns] 35kB dask.array<chunksize=(4387,), meta=np.ndarray>
    ingested_forecast_length            (init_time) timedelta64[ns] 35kB dask.array<chunksize=(4387,), meta=np.ndarray>
    spatial_ref                         int64 8B ...
    valid_time                          (init_time, lead_time) datetime64[ns] 7MB dask.array<chunksize=(4387, 209), meta=np.ndarray>
Data variables:
    pressure_reduced_to_mean_sea_level  (init_time, lead_time, latitude, longitude) float32 59MB dask.array<chunksize=(1, 105, 4, 4), meta=np.ndarray>
    relative_humidity_2m                (init_time, lead_time, latitude, longitude) float32 59MB dask.array<chunksize=(1, 105, 4, 4), meta=np.ndarray>
    total_cloud_cover_atmosphere        (init_time, lead_time, latitude, longitude) float32 59MB dask.array<chunksize=(1, 105, 4, 4), meta=np.ndarray>
Attributes:
    dataset_id:           noaa-gfs-forecast
    dataset_version:      0.2.7
    name:                 NOAA GFS forecast
    description:          Weather forecasts from the Global Forecast System (...
    attribution:          NOAA NWS NCEP GFS data processed by dynamical.org f...
    spatial_domain:       Global
    spatial_resolution:   0.25 degrees (~20km)
    time_domain:          Forecasts initialized 2021-05-01 00:00:00 UTC to Pr...
    time_resolution:      Forecasts initialized every 6 hours
    forecast_domain:      Forecast lead time 0-384 hours (0-16 days) ahead
    forecast_resolution:  Forecast step 0-120 hours: hourly, 123-384 hours: 3...

In [28]:
with DaskClient():
    ds_gfs_subset.to_zarr("data/gfs_benson.zarr", mode="a")

/Users/Shared/working/projects/ml_weather_forecasting/.venv/lib/python3.13/site-packages/zarr/api/asynchronous.py:244: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


In [34]:
# pressure_reduced_to_mean_sea_level, relative_humidity_2m, total_cloud_cover_atmosphere

xr.open_zarr("data/gfs_benson.zarr")#.total_cloud_cover_atmosphere.isel(init_time=-1, latitude=-1, longitude=-1).plot()

<xarray.Dataset> Size: 360MB
Dimensions:                             (init_time: 4387, lead_time: 209,
                                         latitude: 4, longitude: 4)
Coordinates:
  * init_time                           (init_time) datetime64[ns] 35kB 2022-...
  * lead_time                           (lead_time) timedelta64[ns] 2kB 00:00...
  * latitude                            (latitude) float64 32B 52.0 ... 51.25
  * longitude                           (longitude) float64 32B -1.5 ... -0.75
    expected_forecast_length            (init_time) timedelta64[ns] 35kB dask.array<chunksize=(4387,), meta=np.ndarray>
    ingested_forecast_length            (init_time) timedelta64[ns] 35kB dask.array<chunksize=(4387,), meta=np.ndarray>
    spatial_ref                         int64 8B ...
    valid_time                          (init_time, lead_time) datetime64[ns] 7MB dask.array<chunksize=(4387, 209), meta=np.ndarray>
Data variables:
    pressure_reduced_to_mean_sea_level  (init_time, lead_time, latitude, longitude) float32 59MB dask.array<chunksize=(1, 105, 4, 4), meta=np.ndarray>
    relative_humidity_2m                (init_time, lead_time, latitude, longitude) float32 59MB dask.array<chunksize=(1, 105, 4, 4), meta=np.ndarray>
    temperature_2m                      (init_time, lead_time, latitude, longitude) float32 59MB dask.array<chunksize=(1, 105, 4, 4), meta=np.ndarray>
    total_cloud_cover_atmosphere        (init_time, lead_time, latitude, longitude) float32 59MB dask.array<chunksize=(1, 105, 4, 4), meta=np.ndarray>
    wind_u_10m                          (init_time, lead_time, latitude, longitude) float32 59MB dask.array<chunksize=(1, 105, 4, 4), meta=np.ndarray>
    wind_v_10m                          (init_time, lead_time, latitude, longitude) float32 59MB dask.array<chunksize=(1, 105, 4, 4), meta=np.ndarray>
Attributes:
    dataset_id:           noaa-gfs-forecast
    dataset_version:      0.2.7
    name:                 NOAA GFS forecast
    description:          Weather forecasts from the Global Forecast System (...
    attribution:          NOAA NWS NCEP GFS data processed by dynamical.org f...
    spatial_domain:       Global
    spatial_resolution:   0.25 degrees (~20km)
    time_domain:          Forecasts initialized 2021-05-01 00:00:00 UTC to Pr...
    time_resolution:      Forecasts initialized every 6 hours
    forecast_domain:      Forecast lead time 0-384 hours (0-16 days) ahead
    forecast_resolution:  Forecast step 0-120 hours: hourly, 123-384 hours: 3...

## ECMWF - IFS

Data starts 2023-01-18

- url to 20240131 00z = [yyyymmdd]/[run]z/0p4-beta/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2
- url to 20240228 00z = [yyyymmdd]/[run]z/0p25/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2
- url to 20250123 18z = [yyyymmdd]/[run]z/[ifs|aifs]/0p25/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2
- url to 20250209 06z = [yyyymmdd]/[run]z/[ifs]/0p25/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2    * missing AIFS *
- url to 20250225 00z = [yyyymmdd]/[run]z/[ifs|aifs-single]/0p25/[experimental]/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2     * Experimental for AIFS *
- url to 20250227 18z =[yyyymmdd]/[run]z/[ifs]/0p25/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2    * missing AIFS *
- url to now = [yyyymmdd]/[run]z/[ifs|aifs-single]/0p25/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2

In [ ]:
dask_client = DaskClient(n_workers=2, threads_per_worker=2)

def request_url_process_data(url, file_path, date_str, run, step):
    response = requests.get(url)

    if response.status_code == 200:
        with open(file_path, "wb") as f:
            f.write(response.content)
        print(f"Downloaded: {file_path}")

    else:
        print(f"Couldn't download {date_str} {run}z {step}h - Status code: {response.status_code}")
        return False
    


In [ ]:
url_base = "https://ecmwf-forecasts.s3.eu-central-1.amazonaws.com"

date = dt.datetime(2023, 1, 18)
date_str = date.strftime("%Y%m%d")
runs = ["00", "12"]
steps = list(range(0, 121, 3))

futures = []
ds_list = []
for run in runs:
    date_folder = f"data/ecmwf_ifs/{date.strftime('%Y%m%d')}_{run}z"
    os.makedirs(date_folder, exist_ok=True)

    for step in steps:
        url = f"{url_base}/{date_str}/{run}z/0p4-beta/oper/{date_str}{run}0000-{step}h-oper-fc.grib2"
        file_path = os.path.join(date_folder, f"ifs_{date_str}_{run}z_{step}h.grib2")

        future = dask_client.submit(request_url_process_data, url, file_path, date_str, run, step)
        futures.append(future)

wait(futures)

In [ ]:
xr.open_dataset("data/ecmwf_ifs/20230118/ifs_20230118_00z_0h.grib2", engine="cfgrib")

In [ ]:
ec_client = ECMWFClient(source="aws", resol="0p4-beta")

date = "2023-01-18"
steps = list(range(0, 121, 3))

result = ec_client.retrieve(
    date=date,
    time=00,
    step=steps,
    param=["2t", "2d", "msl", "tcc", "10u", "10v"],
    target=f"data/ifs_{date}.grib2",
)

In [ ]:
ds_list = cfgrib.open_datasets